In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from subplots_from_axsize import subplots_from_axsize

import sys
sys.path.insert(0, '..') # in order to be able to import from scripts.py

from scripts.client import VisAVisClient
from scripts.make_protocol import make_protocol
from scripts.entropy_utils import conditional_entropy_discrete
from scripts.plot_result import plot_result

In [ ]:
PARAMETERS_DEFAULT = {
  "c_rate": 1,
  "e_incr": 1,
  "i_incr": 1,
  "r_incr": 0.0667
}

In [ ]:
def make_binary_protocol(interval, n_bits, p=0.5):
    pulse_bits = np.random.choice(2, size=n_bits, p=[1-p, p])
    pulse_bits[0] = 1

    locs_1, = np.nonzero(pulse_bits)

    pulse_times = interval * locs_1
    pulse_intervals = pulse_times[1:] - pulse_times[:-1]
    
    return pulse_times, pulse_intervals

In [ ]:
from scipy.signal import find_peaks

def results_to_arrival_times(
    result,
):
    h_max = result.states['h'].max()
    data = result.states[result.states['h'] == h_max].copy()
    
    data['act'] = 1.0 * ((data['E'] > 0) | (data['I'] > 0))
    data = data.groupby(['seconds', 'h'])['act'].mean()
    
    h_max = result.states['h'].max()
    
    output_series = data[:, h_max]
    arrival_idxs, _ = find_peaks(output_series, distance=5)
    arrival_times = output_series.index[arrival_idxs]
    
    return arrival_times

In [ ]:
def construct_dataset(
    interval,
    pulse_times,
    arrival_times,
    offset=720, # how should this be determined?
    n_nearest=1,
):
    t0 = 0
    xs = []
    yss = []
    while t0 <= max(pulse_times):
        t0 += interval
        
        # was it a pulse?
        x = t0 in pulse_times

        # predicted arrival time
        t1 = t0 + offset

        # find predicted arrival time in actual arrival times
        idx = np.searchsorted(arrival_times, t1)

        # if there is not enough pulses before/after don't add to dataset
        if idx < n_nearest or idx > len(arrival_times) - n_nearest:
            continue

        # subtract predicted arrival time!
        ys = arrival_times[idx-n_nearest:idx+n_nearest] - t1

        xs.append(x)
        yss.append(ys)

    xs = np.array(xs)
    yss = np.array(yss)
    return xs, yss

In [ ]:
from tqdm.notebook import tqdm

def generate_dataset(
    interval,
    n_bits,
    n_simulations,
    channel_height=7,
    duration=5,
    n_margin=5,
    offset=720, # how should this be determined?
):
    client = VisAVisClient(
        visavis_bin=f'../target/bins/vis-a-vis-{channel_height}',
    )
    
    data_parts = []
    for simulation_id in tqdm(range(n_simulations)):
        pulse_times, pulse_intervals = make_binary_protocol(interval, n_bits, p=0.5)
        
        protocol_file_path = make_protocol(
            pulse_intervals=list(pulse_intervals) + [1500],
            duration=duration,
            out_folder='/tmp/',
        )

        result = client.run(
            parameters_json=PARAMETERS_DEFAULT,
            protocol_file_path=protocol_file_path,
            verbose=False,
        )
        
        arrival_times = results_to_arrival_times(result)
        
        xs, yss = construct_dataset(
            interval=interval,
            pulse_times=pulse_times,
            arrival_times=arrival_times,
            n_nearest=1, # just need before and after to get the closest one
        )

        # compute closest pulse
        yss_abs_argmins = np.argmin(np.abs(yss), axis=1, keepdims=True)
        cs = np.take_along_axis(yss, yss_abs_argmins, axis=1)[:, 0]
        
        # drop margins to increase quality of samples
        xs = xs[n_margin:-n_margin]
        cs = cs[n_margin:-n_margin]
        
        data_part = pd.DataFrame({'x': xs, 'c': cs})
        data_part['simulation_id'] = simulation_id
        data_parts.append(data_part)
        
    return pd.concat(data_parts)

In [ ]:
def plot_hist(data, outfile=None):
    fig, ax = subplots_from_axsize(1, 1, (4, 3), left=0.8)

    ax.hist(
        data[~data['x']]['c'],
        color='red', alpha=0.3, bins=np.linspace(-600, 600, 81), density=True
    )

    ax.hist(
        data[data['x']]['c'],
        color='green', alpha=0.3, bins=np.linspace(-600, 600, 81), density=True
    )

    ax.set_xlabel('time to closest arrival')
    ax.set_ylabel('pdf')
    ax.set_ylim(0, 0.025)

    if outfile is not None:
        plt.savefig(outfile, dpi=200)
    else:
        return fig, ax

In [ ]:
def plot_entropy_by_n_neighbours(data, outfile=None, k_low=3, k_high=51):
    ks = np.arange(k_low, k_high, 1)

    es = np.array([
        conditional_entropy_discrete(
            data['x'].to_numpy(),
            data['c'].to_numpy().reshape(-1, 1),
            n_neighbors=k
        )
        for k in ks
    ])
    
    fig, ax = subplots_from_axsize(1, 1, (4, 3), left=0.8)

    ax.plot(
        ks, es, '-ok'
    )

    ax.set_ylim(0, 1)
    ax.set_xlabel('n_neighbours')
    ax.set_ylabel('H(X | distance to closes arrival)')
    
    if outfile is not None:
        plt.savefig(outfile, dpi=200)

    return ks, es

# run

In [ ]:
from pathlib import Path

outdir = Path('../private/binary_attempt_2023-09-19')

In [ ]:
%%time

data_parts = []

for channel_height in [4, 5, 6, 7, 8, 9, 10]:
    for interval in [80, 85, 90, 95, 100, 105, 110, 115, 120, 130, 150, 180]:
        data = generate_dataset(
            interval=interval,
            channel_height=channel_height,
            n_bits=100,
            n_simulations=25,
            duration=5,
        )

        data['channel_height'] = channel_height
        data['interval'] = interval
        
        data_parts.append(data)
        data_all = pd.concat(data_parts)
        data_all.to_csv(outdir / 'data_all.csv')

In [ ]:
%%time

results = []

for (channel_height, interval), data in data_all.groupby(['channel_height', 'interval']):
    cond_entropy = min([
        conditional_entropy_discrete(
            data['x'].to_numpy(),
            data['c'].to_numpy().reshape(-1, 1),
            n_neighbors=k
        )
        for k in np.arange(10, 51, 1)
    ])
    
    mi_tick = 1 - cond_entropy
    bitrate_per_sec = mi_tick / interval
    bitrate_per_hour = bitrate_per_sec * 60 * 60
    
    results.append({
        'channel_height': channel_height,
        'interval': interval,
        'cond_entropy': cond_entropy,
        'mi_tick': mi_tick,
        'bitrate_per_hour': bitrate_per_hour,
    })
    
results = pd.DataFrame(results)

In [ ]:
intervals = np.unique(results['interval'])

fig, ax = subplots_from_axsize(1, 1, (5, 4), left=0.7)

for channel_height, results_h in results.groupby('channel_height'):
    ax.plot(
        results_h['interval'],
        results_h['bitrate_per_hour'],
        '-o',
        label=f'h = {channel_height}',

    )
    
ax.plot(
    intervals,
    60 * 60 / intervals,
    ':',
    label=f'perfect',
)
    
ax.set_xlabel('interval')
ax.set_ylabel('bitrate [bit/hour]')
ax.set_ylim(0, 30)

ax.xaxis.set_minor_locator(mticker.MultipleLocator(10))
ax.yaxis.set_minor_locator(mticker.MultipleLocator(2.5))
ax.grid(which='both', ls=':')

ax.legend()

fig.savefig(outdir / 'bitrates.png')

In [ ]:
intervals = np.unique(results['interval'])

fig, ax = subplots_from_axsize(1, 1, (5, 4), left=0.7)

for channel_height, results_h in results.groupby('channel_height'):
    ax.plot(
        results_h['interval'],
        results_h['mi_tick'],
        '-o',
        label=f'h = {channel_height}',

    )
    
ax.plot(
    intervals,
    np.ones_like(intervals),
    ':',
    label=f'perfect',
)
    
ax.set_xlabel('interval')
ax.set_ylabel('MI per tick [bit]')
ax.set_ylim(0.0, 1.03)

ax.xaxis.set_minor_locator(mticker.MultipleLocator(10))
ax.yaxis.set_minor_locator(mticker.MultipleLocator(0.1))
ax.grid(which='both', ls=':')

ax.legend()

fig.savefig(outdir / 'mi_per_ticks.png')